In [37]:
# ###########################################################################
# # File: llama_cpp_quantization.ipynb
# # Author: [NeuraFusionAI]
# # Created: [Aug 11 2024]
# #
# # Description:
# # This script/notebook implements quantization techniques for Llama models
# # using C++ backend. It aims to reduce model size and inference latency
# # while maintaining accuracy. The script is designed to be modular and
# # easily integrable into larger AI systems.
# #
# # Version: 1.0
# # Changelog:
# # - v1.0: Initial implementation of the quantization pipeline.
# ###########################################################################

Download Model from HuggingFace

In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
from huggingface_hub import snapshot_download

model_name = "meta-llama/Meta-Llama-3.1-8B"
base_model = "./original_model/"
snapshot_download(repo_id=model_name, local_dir=base_model, ignore_patterns=["*.pth"])

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

USE_POLICY.md:   0%|          | 0.00/4.69k [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/2.18M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

'/content/original_model'

Clone llama.cpp Repository

In [4]:
!git clone https://github.com/ggerganov/llama.cpp

Cloning into 'llama.cpp'...
remote: Enumerating objects: 31876, done.
remote: Counting objects: 100% (10209/10209), done.
remote: Compressing objects: 100% (701/701), done.
remote: Total 31876 (delta 9921), reused 9545 (delta 9502), pack-reused 21667
Receiving objects: 100% (31876/31876), 55.33 MiB | 23.53 MiB/s, done.
Resolving deltas: 100% (22963/22963), done.


In [5]:
!mkdir models

Convert model to GGUF format

In [6]:
!python llama.cpp/convert_hf_to_gguf.py ./original_model/ --outfile models/llama_3.1_FP16.gguf

INFO:hf-to-gguf:Loading model: original_model
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to-gguf:gguf: loading model part 'model-00001-of-00004.safetensors'
INFO:hf-to-gguf:token_embd.weight,           torch.bfloat16 --> F16, shape = {4096, 128256}
INFO:hf-to-gguf:blk.0.attn_norm.weight,      torch.bfloat16 --> F32, shape = {4096}
INFO:hf-to-gguf:blk.0.ffn_down.weight,       torch.bfloat16 --> F16, shape = {14336, 4096}
INFO:hf-to-gguf:blk.0.ffn_gate.weight,       torch.bfloat16 --> F16, shape = {4096, 14336}
INFO:hf-to-gguf:blk.0.ffn_up.weight,         torch.bfloat16 --> F16, shape = {4096, 14336}
INFO:hf-to-gguf:blk.0.ffn_norm.weight,       torch.bfloat16 --> F32, shape = {4096}
INFO:hf-to-gguf:blk.0.attn_k.weight,         torch.bfloat16 --> F16, shape = {4096, 1024}
INFO:hf-to-gguf:blk.0.attn_output.weight,    torch.bfloat16 --> F16,

Build llama.cpp and quantize the Model

In [7]:
!mkdir llama.cpp/build && cd llama.cpp/build && cmake .. && cmake --build . --config Release

-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found Git: /usr/bin/git (found version "2.34.1")
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD - Success
-- Found Threads: TRUE
-- Found OpenMP_C: -fopenmp (found version "4.5")
-- Found OpenMP_CXX: -fopenmp (found version "4.5")
-- Found OpenMP: TRUE (found version "4.5")
-- OpenMP found
-- Using llamafile
-- Warning: ccache not found - consider installing it for faster compilation or disable this warning with GGML_CCACHE=OFF
-- CMAKE_SYSTEM_PROCE

In [8]:
!cd llama.cpp/build/bin && ./llama-quantize /content/models/llama_3.1_FP16.gguf /content/models/llama_3.1-Q4_K_M.gguf q4_K_M

main: build = 3569 (8cd1bcfd)
main: built with cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0 for x86_64-linux-gnu
main: quantizing '/content/models/llama_3.1_FP16.gguf' to '/content/models/llama_3.1-Q4_K_M.gguf' as Q4_K_M
llama_model_loader: loaded meta data with 26 key-value pairs and 292 tensors from /content/models/llama_3.1_FP16.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Original_Model
llama_model_loader: - kv   3:                         general.size_label str              = 8.0B
llama_model_loader: - kv   4:                            general.license str              = llama3.1
llama_model_loader: - kv   5:          

Now Inference using Quantized Model

In [9]:
!pip install llama-cpp-python==0.2.85

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 MB 12.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.2 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.85-cp310-cp310-linux_x86_64.whl size=2873169 sha256=a813803780a383c7ec10b32994cdfdbfcad72fdec02ab006a38b7cd4851ee012
  Stored in directory: /root/.cache/pip/wheels/3f/e8/4e/29a754f9175ef52b6481cd75e3af4de38bf6dfa9c2972f75d4
Successfully built llama-cpp-python


In [10]:
from llama_cpp import Llama

In [11]:
model_path = "/content/models/llama_3.1-Q4_K_M.gguf"

In [12]:
llm = Llama(model_path=model_path)

llama_model_loader: loaded meta data with 26 key-value pairs and 292 tensors from /content/models/llama_3.1-Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Original_Model
llama_model_loader: - kv   3:                         general.size_label str              = 8.0B
llama_model_loader: - kv   4:                            general.license str              = llama3.1
llama_model_loader: - kv   5:                               general.tags arr[str,6]       = ["facebook", "meta", "pytorch", "llam...
llama_model_loader: - kv   6:                          general.languages arr[str,8]       = ["en", "de", "fr", "it", "pt"

In [13]:
generation_kwargs = {
    "max_tokens":200,
    "echo":False,
    "top_k":1
}

prompt = "Which country hosted 2018 fifa world cup?"
res = llm(prompt, **generation_kwargs)
res


llama_print_timings:        load time =    7601.18 ms
llama_print_timings:      sample time =      19.30 ms /   200 runs   (    0.10 ms per token, 10360.01 tokens per second)
llama_print_timings: prompt eval time =    7601.10 ms /    12 tokens (  633.42 ms per token,     1.58 tokens per second)
llama_print_timings:        eval time =  142414.35 ms /   199 runs   (  715.65 ms per token,     1.40 tokens per second)
llama_print_timings:       total time =  150359.93 ms /   211 tokens


{'id': 'cmpl-98c4db2f-8b80-4618-b224-df05620641c5',
 'object': 'text_completion',
 'created': 1723399330,
 'model': '/content/models/llama_3.1-Q4_K_M.gguf',
 'choices': [{'text': " The 2018 fifa world cup was the 21st fifa world cup, an international football tournament contested by the men's national teams of the member associations of fifa. It took place in russia from 14 june to 15 july 2018. It was the first world cup to be held in eastern europe, and the 11th time that it had been held in europe. At an estimated cost of over $14.2 billion, it. The 2018 fifa world cup was the 21st fifa world cup, an international football tournament contested by the men's national teams of the member associations of fifa. It took place in russia from 14 june to 15 july 2018. It was the first world cup to be held in eastern europe, and the 11th time that it had been held in europe. At an estimated cost of over $14.2 billion, it. The 2018",
   'index': 0,
   'logprobs': None,
   'finish_reason': 'len

Save model to Google Drive

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
!mkdir "/content/drive/My Drive/llama_models"

mkdir: cannot create directory ‘/content/drive/My Drive/llama_models’: File exists


In [16]:
import shutil

source_file_path = '/content/models/llama_3.1-Q4_K_M.gguf'
destination_file_path = '/content/drive/My Drive/llama_models/llama_3.1-Q4_K_M.gguf'

shutil.copy(source_file_path, destination_file_path)

'/content/drive/My Drive/llama_models/llama_3.1-Q4_K_M.gguf'

Upload model to Huggingface Hub

In [19]:
from huggingface_hub import login
login('token')

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [23]:
from huggingface_hub import HfApi
api = HfApi()

model_id = "NeuraFusionAI/llama3.1-Q4_K_M-gguf"
api.create_repo(model_id, exist_ok=True, repo_type="model")
api.upload_file(
    path_or_fileobj='/content/models/llama_3.1-Q4_K_M.gguf',
    path_in_repo="llama3.1-Q4_K_M.gguf",
    repo_id=model_id,
)

llama_3.1-Q4_K_M.gguf:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/NeuraFusionAI/llama3.1-Q4_K_M-gguf/commit/d6c57aa6e5523f86d048618cb97402cc782b10dc', commit_message='Upload llama3.1-Q4_K_M.gguf with huggingface_hub', commit_description='', oid='d6c57aa6e5523f86d048618cb97402cc782b10dc', pr_url=None, pr_revision=None, pr_num=None)

Inference by utilizing GPU

In [26]:
from huggingface_hub import snapshot_download

model_name = "NeuraFusionAI/llama3.1-Q4_K_M-gguf"
base_model = "./quantized_models/"
snapshot_download(repo_id=model_name, local_dir=base_model)

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

llama3.1-Q4_K_M.gguf:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

'/content/quantized_models'

In [27]:
!nvidia-smi

Sun Aug 11 18:25:01 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [28]:
!pip install llama-cpp-python --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu122

Looking in indexes: https://pypi.org/simple, https://abetlen.github.io/llama-cpp-python/whl/cu122


In [33]:
# Correct import statement
from llama_cpp import Llama

# Correct model path and initialization
model_path = "./quantized_models/llama3.1-Q4_K_M.gguf"

# Initialize the Llama model
# n_gpu_layers is set to -1, which might indicate that the model should fully utilize the GPU if available
model = Llama(model_path=model_path, n_gpu_layers=-1)


llama_model_loader: loaded meta data with 26 key-value pairs and 292 tensors from ./quantized_models/llama3.1-Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Original_Model
llama_model_loader: - kv   3:                         general.size_label str              = 8.0B
llama_model_loader: - kv   4:                            general.license str              = llama3.1
llama_model_loader: - kv   5:                               general.tags arr[str,6]       = ["facebook", "meta", "pytorch", "llam...
llama_model_loader: - kv   6:                          general.languages arr[str,8]       = ["en", "de", "fr", "it", "p

In [34]:
generation_kwargs = {
    "max_tokens":200,
    "echo":False,
    "top_k":1
}

prompt = "Which country hosted 2018 fifa world cup?"
res = model(prompt, **generation_kwargs)
res


llama_print_timings:        load time =    4790.91 ms
llama_print_timings:      sample time =      19.31 ms /   200 runs   (    0.10 ms per token, 10358.94 tokens per second)
llama_print_timings: prompt eval time =    4790.84 ms /    12 tokens (  399.24 ms per token,     2.50 tokens per second)
llama_print_timings:        eval time =  139957.63 ms /   199 runs   (  703.30 ms per token,     1.42 tokens per second)
llama_print_timings:       total time =  145182.47 ms /   211 tokens


{'id': 'cmpl-6d84175b-47fc-4e11-b503-5810c26e3ac5',
 'object': 'text_completion',
 'created': 1723400937,
 'model': './quantized_models/llama3.1-Q4_K_M.gguf',
 'choices': [{'text': " The 2018 fifa world cup was the 21st fifa world cup, an international football tournament contested by the men's national teams of the member associations of fifa. It took place in russia from 14 june to 15 july 2018. It was the first world cup to be held in eastern europe, and the 11th time that it had been held in europe. At an estimated cost of over $14.2 billion, it. The 2018 fifa world cup was the 21st fifa world cup, an international football tournament contested by the men's national teams of the member associations of fifa. It took place in russia from 14 june to 15 july 2018. It was the first world cup to be held in eastern europe, and the 11th time that it had been held in europe. At an estimated cost of over $14.2 billion, it. The 2018",
   'index': 0,
   'logprobs': None,
   'finish_reason': 'l

In [35]:
output = model("Provide Information about world war 2 in 1000 words.", max_tokens=2048, stop=["\n"], echo=False)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    4790.91 ms
llama_print_timings:      sample time =       0.59 ms /     6 runs   (    0.10 ms per token, 10152.28 tokens per second)
llama_print_timings: prompt eval time =    6536.73 ms /    13 tokens (  502.83 ms per token,     1.99 tokens per second)
llama_print_timings:        eval time =    3228.44 ms /     5 runs   (  645.69 ms per token,     1.55 tokens per second)
llama_print_timings:       total time =    9775.95 ms /    18 tokens


In [36]:
print(output['choices'][0]['text'])

 Also provide 5 sources.


In [38]:
# Note: The quantization process is designed to scale with model size.
# Larger models will benefit more from the reduction in memory usage,
# allowing deployment on lower-spec hardware.

In [39]:
# Warning: Ensure that the backend supports 8-bit quantization.
# Some hardware may not be compatible with lower precision calculations.